# Update metadata using information in table_meta.csv

* update location field in table_meta.csv with information in updated_location_Theresa_Keates_june2020
* update netcdf files

In [1]:
from pathlib import Path
import sys
import os
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from IPython.display import display,Image

path_MEOP_analyses = os.fspath(Path.home() / 'MEOP_analyses/')
if not (path_MEOP_analyses in sys.path):
        sys.path.insert(0,path_MEOP_analyses)

import importlib
import meop
importlib.reload(meop)

def copy_file(file_name,src_dir,dst_dir):
    shutil.copyfile(Path(src_dir)/file_name,Path(dst_dir)/file_name)

In [80]:
meta = pd.read_csv(meop.processdir / 'table_meta.csv').set_index('smru_platform_code')

In [81]:
meta_update = pd.read_csv(Path('Theresa_Keates_NPac') / 'updated_location_Theresa_Keates_june2020.csv', usecols = ['tag','location'])
meta_update.columns = ['smru_platform_code','location']
meta_update = meta_update.set_index('smru_platform_code')

In [132]:
meta_combined = meta_update.combine_first(meta)
meta_combined

,location,reference_doi,species
smru_platform_code,,,
DL02-C400-16,NaN,NaN,Grey seal
DL02-C401-16,NaN,NaN,Grey seal
awru1-A-06,Davis,NaN,Weddell seal
awru1-B-06,Davis,NaN,Weddell seal
awru1-C-06,Davis,NaN,Weddell seal
...,...,...,...
wd1-10216-06,Davis,NaN,Weddell seal
wd1-10217-06,Davis,NaN,Weddell seal
wd1-9497-06,Davis,NaN,Weddell seal


In [133]:
meta_combined.to_csv(meop.processdir / 'table_meta.csv')

# Update metadata in ncfiles

In [160]:
def update_metadata(path_meta = meop.processdir / 'table_meta.csv', EXP='',one_smru_name=''):

    import netCDF4

    df_meta = pd.read_csv(path_meta).set_index('smru_platform_code')

    if one_smru_name in df_meta.index:
        df_meta = df_meta.loc[[one_smru_name],:]
    elif EXP:
        df_meta['EXP'] = df_meta.index
        df_meta['EXP'] = df_meta.EXP.str.split('-').apply(lambda x: x[0])
        df_meta = df_meta.loc[df_meta.EXP == EXP,:]

    modes = ['lr0','lr1','hr0','hr1','fr0','fr1']
    for tag in df_meta.index:

        meta_row = df_meta.loc[tag,:].dropna()

        for qf in modes:

            name_prof = meop.fname_prof(tag,qf=qf)        
            if Path(name_prof).exists():

                with netCDF4.Dataset(name_prof,'a') as f:
                    for col in meta_row.keys():
                        if f.location != meta_row[col]:
                            f.location = meta_row[col]

    return

update_metadata()